<img src="../../../images/qiskit_header.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" align="middle">

# Non-Clifford Randomized Benchmarking

- Last Updated: ???
- Requires: qiskit-terra ???, qiskit-ignis ???, qiskit-aer ???

## Introduction

**Non-Clifford Randomized Benhcmaking** is a variant of the Randomized Benchmarking (RB) method that is used for benchmarking certain non-Clifford gates via randomization, taken from the so-called *CNOT-dihedral group* which is generated by CNOTs and a single qubit dihedral group $\langle T, X \rangle$.
The method is based on the paper "Scalable randomized benchmarking of non-Clifford gates"(https://arxiv.org/abs/1510.02720).

This notebook gives an example for how to use the ``ignis.verification.randomized_benchmarking`` module in order to perform interleaved RB.

In [1]:
#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

import os, sys
HOME = "/home/shelly/Quantum/ignis2"
QISKIT_ROOT = HOME
root_dir = os.path.expanduser(QISKIT_ROOT)
sys.path = [os.path.expanduser(QISKIT_ROOT)] + sys.path
HOME = "/home/shelly/Quantum/qiskit-terra"
QISKIT_ROOT = HOME
root_dir = os.path.expanduser(QISKIT_ROOT)
sys.path = [os.path.expanduser(QISKIT_ROOT)] + sys.path
print (sys.path)

import qiskit.ignis.verification.randomized_benchmarking as rb
print(rb.__file__)

#Import Qiskit classes classes
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

['/home/shelly/Quantum/qiskit-terra', '/home/shelly/Quantum/ignis2', '/opt/anaconda3/envs/ShellyEnv/lib/python36.zip', '/opt/anaconda3/envs/ShellyEnv/lib/python3.6', '/opt/anaconda3/envs/ShellyEnv/lib/python3.6/lib-dynload', '', '/opt/anaconda3/envs/ShellyEnv/lib/python3.6/site-packages', '/opt/anaconda3/envs/ShellyEnv/lib/python3.6/site-packages/IPython/extensions', '/home/shelly/.ipython']
/home/shelly/Quantum/ignis2/qiskit/ignis/verification/randomized_benchmarking/__init__.py


## Select the Parameters of the non-Clifford RB Run

First, we need to choose the regular RB parameters:

- **nseeds:** The number of seeds. For each seed there you will get a separate list of output circuits in rb_circs.
- **length_vector:** The length vector of CNOT-dihedral lengths. Must be in ascending order. RB sequences of increasing length grow on top of the previous sequences.
- **rb_pattern:** A list of the form [[i,j],[k],...] which will make simultaneous RB sequences where Qi,Qj are a 2-qubit RB sequence and Qk is a 1-qubit sequence, etc. The number of qubits is the sum of the entries. For 'regular' RB the qubit_pattern is just [[0]],[[0,1]].
- **length_multiplier:** If this is an array it scales each rb_sequence by the multiplier.
- **seed_offset:** What to start the seeds at (e.g. if we want to add more seeds later)
- **align_cliffs:** If true adds a barrier across all qubits in rb_pattern after each set of cliffords

As well as another parameter for non-Clifford RB:

- **group_gates = 'Non-Clifford'** 

In this example we have 3 qubits Q0,Q1,Q2. We are running 2Q RB (on qubits Q0,Q2) and 1Q RB (on qubit Q1) simultaneously, where there are three times as many 1Q gates.

In [2]:
#Number of qubits
nQ = 2
#There are 3 qubits: Q0,Q1.
#Number of seeds (random sequences)
nseeds = 5
#Number of Cliffords in the sequence (start, stop, steps)
nCliffs = np.arange(1,200,20)
#2Q RB on Q0,Q2 and 1Q RB on Q1
rb_pattern = [[0,1]]
# Non-Clifford RB
group_gates = 'Non-Clifford'

## Generate non-Clifford RB sequences

We generate RB sequences. We start with a small example (so it doesn't take too long to run).

In order to generate the non-Clifford RB sequences rb_nonclifford_Z_circs and rb_nonclifford_X_circs, which are a list of lists of quantum circuits, we run the function **rb.randomized_benchmarking_seq**.

This function returns:

- **rb_nonclifford_Z_circs:** A list of lists of circuits for the standard rb sequences where we measure the $|0 \dots 0 \rangle$ state (separate list for each seed).
- **xdata:** The CNOT-dihedral group elements lengths (with multiplier if applicable).
- **rb_nonclifford_X_circs:** A list of lists of circuits for the rb sequences where we measure the $|+ \dots + \rangle$ state (separate list for each seed).

In [3]:
rb_opts = {}
rb_opts['length_vector'] = nCliffs
rb_opts['nseeds'] = nseeds
rb_opts['rb_pattern'] = rb_pattern
rb_opts['group_gates'] = group_gates
rb_opts['align_cliffs'] = True

CS01 = rb.dihedral.CNOTDihedral(2)
CS01.phase(1, 0)
CS01.phase(1, 1)
CS01.cnot(0, 1)
CS01.phase(7, 1)
CS01.cnot(0, 1)
rb_opts['interleaved_gates'] = [CS01]

rb_nonclifford_Z_circs, xdata, rb_nonclifford_X_circs, rb_nonclifford_interleaved_Z_circs, \
rb_nonclifford_interleaved_X_circs = rb.randomized_benchmarking_seq(**rb_opts)

TypeError: 'CNOTDihedral' object does not support indexing

As an example, we print the circuit corresponding to the first RB sequences:

(Note that rb_nonclifford_X_circs is the same as rb_nonclifford_X_circs except of the Hadamrd gates at the beggining anf the end of the sequence).

In [ ]:
print (rb_nonclifford_Z_circs[0][0])

In [ ]:
print (rb_nonclifford_interleaved_Z_circs[0][0])

In [ ]:
print (rb_nonclifford_X_circs[0][0])

In [ ]:
print (rb_nonclifford_interleaved_X_circs[0][0])

## Define the noise model

We define a noise model for the simulator. To simulate decay, we add depolarizing error probabilities to the CNOT and U gates.

In [ ]:
noise_model = NoiseModel()
p1Q = 0.002
p2Q = 0.01
noise_model.add_all_qubit_quantum_error(depolarizing_error(p1Q, 1), 'u2')
noise_model.add_all_qubit_quantum_error(depolarizing_error(2*p1Q, 1), 'u3')
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2Q, 2), 'cx')

We can execute the RB sequences either using Qiskit Aer Simulator (with some noise model) or using IBMQ provider, and obtain two lists of results.

In [ ]:
#Non-Clifford RB X circuits
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
nonclifford_X_result_list = []
nonclifford_X_qobj_list = []
import time
for rb_seed,rb_circ_seed in enumerate(rb_nonclifford_X_circs):
    print('Compiling seed %d'%rb_seed)
    new_rb_circ_seed = qiskit.compiler.transpile(rb_circ_seed, basis_gates=basis_gates)
    qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
    print('Simulating seed %d'%rb_seed)
    job = backend.run(qobj, noise_model=noise_model, backend_options={'max_parallel_experiments': 0})
    nonclifford_X_result_list.append(job.result())
    nonclifford_X_qobj_list.append(qobj)    
print("Finished Simulating Non-Clifford X Circuits")

In [ ]:
#Non-Clifford interleaved RB X circuits
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
nonclifford_interleaved_X_result_list = []
nonclifford_interleaved_X_qobj_list = []
import time
for rb_seed,rb_circ_seed in enumerate(rb_nonclifford_interleaved_X_circs):
    print('Compiling seed %d'%rb_seed)
    new_rb_circ_seed = qiskit.compiler.transpile(rb_circ_seed, basis_gates=basis_gates)
    qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
    print('Simulating seed %d'%rb_seed)
    job = backend.run(qobj, noise_model=noise_model, backend_options={'max_parallel_experiments': 0})
    nonclifford_interleaved_X_result_list.append(job.result())
    nonclifford_interleaved_X_qobj_list.append(qobj)    
print("Finished Simulating Non-Clifford X Circuits")

In [ ]:
#Non-Clifford RB Z circuits
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
nonclifford_Z_result_list = []
nonclifford_Z_qobj_list = []
import time
for rb_seed,rb_circ_seed in enumerate(rb_nonclifford_Z_circs):
    print('Compiling seed %d'%rb_seed)
    new_rb_circ_seed = qiskit.compiler.transpile(rb_circ_seed, basis_gates=basis_gates)
    qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
    print('Simulating seed %d'%rb_seed)
    job = backend.run(qobj, noise_model=noise_model, backend_options={'max_parallel_experiments': 0})
    nonclifford_Z_result_list.append(job.result())
    nonclifford_Z_qobj_list.append(qobj)    
print("Finished Simulating Non-Clifford Z Circuits")

In [ ]:
#Non-Clifford inteleaved RB Z circuits
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
nonclifford_interleaved_Z_result_list = []
nonclifford_interleaved_Z_qobj_list = []
import time
for rb_seed,rb_circ_seed in enumerate(rb_nonclifford_interleaved_Z_circs):
    print('Compiling seed %d'%rb_seed)
    new_rb_circ_seed = qiskit.compiler.transpile(rb_circ_seed, basis_gates=basis_gates)
    qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
    print('Simulating seed %d'%rb_seed)
    job = backend.run(qobj, noise_model=noise_model, backend_options={'max_parallel_experiments': 0})
    nonclifford_interleaved_Z_result_list.append(job.result())
    nonclifford_interleaved_Z_qobj_list.append(qobj)    
print("Finished Simulating Non-Clifford Z Circuits")

## Fit the results

We fit the results of the non-Clifford Z RB circuits and the non-Clifford X RB circuits into an exponentially decaying function and obtain the Errors per group element $\alpha_Z$ and $\alpha_R$ of the Z and X sequences respectively.

In [ ]:
#Create the non-Clifford Z and X RB fitters
nonclifford_Z_rb_fit = rb.RBFitter(nonclifford_Z_result_list, xdata, rb_opts['rb_pattern'])
nonclifford_X_rb_fit = rb.RBFitter(nonclifford_X_result_list, xdata, rb_opts['rb_pattern'])

In [ ]:
#Create the non-Clifford intelreaved Z and X RB fitters
nonclifford_interleaved_Z_rb_fit = rb.RBFitter(nonclifford_interleaved_Z_result_list, xdata, rb_opts['rb_pattern'])
nonclifford_interleaved_X_rb_fit = rb.RBFitter(nonclifford_interleaved_X_result_list, xdata, rb_opts['rb_pattern'])

### Calculate the non-Clifford gate error fidelity

From the values of $\alpha_Z$ and $\alpha_R$ we obtain the depolorizing channnel parameter:
$$ \alpha = (\alpha_Z + 2^n \alpha_R) / (2^n+1)$$
and the average gate error (EPG) of a CNOT-dihedral group element: 
$$r = (2^n-1)(1-\alpha)/2^n .$$

In [ ]:
#Calculate the joint fitter
joint_rb_fit = rb.NonCliffordRBFitter(nonclifford_Z_result_list, nonclifford_X_result_list, xdata, rb_opts['rb_pattern'])
joint_int_rb_fit = rb.NonCliffordRBFitter(nonclifford_interleaved_Z_result_list, nonclifford_interleaved_X_result_list, xdata, rb_opts['rb_pattern'])

In [ ]:
#Print the joint fitter parameters
for patt_ind, pattern in enumerate(rb_pattern):
    print ('pattern:', patt_ind, '-', len(pattern), 'qubit non-Clifford RB:', joint_rb_fit.fit_nonclifford[patt_ind])
#Print the joint interleaved parameters
for patt_ind, pattern in enumerate(rb_pattern):
    print ('pattern:', patt_ind, '-', len(pattern), 'qubit non-Clifford interleaved RB:', joint_int_rb_fit.fit_nonclifford[patt_ind])

In [ ]:
#Expected error of the interleaved gate
alpha = joint_rb_fit.fit_nonclifford[0]['alpha']
alpha_C = joint_int_rb_fit.fit_nonclifford[0]['alpha']
d=4
r_est = (d-1)*(1-alpha_C/alpha)/d
print ("Estimated gate error:", r_est)

### Plot the results

In [ ]:
#Plot the joint RB data
plt.figure(figsize=(15, 6))

for i in range(1):
    ax = plt.subplot(1, 2, i+1)
    pattern_ind = i

    # Plot the essence by calling plot_rb_data
    joint_rb_fit.plot_rb_data(pattern_ind, ax=ax, add_label=True, show_plt=False)

    # Add title and label
    ax.set_title('%d Qubit non-Clifford RB'%(len(rb_opts['rb_pattern'][i])), fontsize=18)

plt.show()

In [ ]:
#Plot the joint interleaved RB data
plt.figure(figsize=(15, 6))

for i in range(1):
    ax = plt.subplot(1, 2, i+1)
    pattern_ind = i

    # Plot the essence by calling plot_rb_data
    joint_int_rb_fit.plot_rb_data(pattern_ind, ax=ax, add_label=True, show_plt=False)

    # Add title and label
    ax.set_title('%d Qubit non-Clifford interleaved RB'%(len(rb_opts['rb_pattern'][i])), fontsize=18)

plt.show()

### Predicted Gate Fidelity

From the known depolarizing errors on the simulation we can predict the **fidelity**. First we need to count the number of gates per a CNOT-dihedral.

The function **gates_per_clifford** takes a compiled qobj and outputs the number of basis gates in each circuit.

(Naturally, in the CNOT-dihedral group there are no u2 gates)

In [ ]:
#Count the number of single and 2Q gates in the 2Q CNOT-dihedral group elemnets
gates_per_elmnt = rb.rb_utils.gates_per_clifford(nonclifford_Z_qobj_list,xdata[0],basis_gates,rb_opts['rb_pattern'][0])
for i in range(len(basis_gates)):
    print("Number of %s gates per CNOT-dihedral element: %f"%(basis_gates[i],
                                                              np.mean([gates_per_elmnt[0][i],gates_per_elmnt[1][i]])))

The **twoQ_clifford_error** gives measured errors in the basis gates that were used to construct the CNOT-dihedral element. It assumes that the error in the underlying gates is depolarizing. It outputs the error per a 2-qubit CNOT-dihedral element.

The input to this function is:

- **ngates:** list of the number of gates per 2Q Clifford
- **gate_qubit:** list of the qubit corresponding to the gate (0, 1 or -1). -1 corresponds to the 2Q gate.
- **gate_err:** list of the gate errors

In [ ]:
#Prepare lists of the number of qubits and the errors
ngates = np.zeros(7)
ngates[0:3] = gates_per_elmnt[0][0:3]
ngates[3:6] = gates_per_elmnt[1][0:3]
ngates[6] = gates_per_elmnt[0][3]
gate_qubits = np.array([0,0,0,1,1,1,-1], dtype=int)
gate_errs = np.zeros(len(gate_qubits))
gate_errs[[1,4]] = p1Q/2 #convert from depolarizing error to epg (1Q)
gate_errs[[2,5]] = 2*p1Q/2 #convert from depolarizing error to epg (1Q)
gate_errs[6] = p2Q*3/4 #convert from depolarizing error to epg (2Q)

#Calculate the predicted EPG
pred_epg = rb.rb_utils.twoQ_clifford_error(ngates,gate_qubits,gate_errs)
print("Predicted 2Q Error per a CNOT-dihedral element: %e"%pred_epg)

In [ ]:
print (rb_nonclifford_Z_circs[0][0][0])
print (rb_nonclifford_X_circs[0][0][0])